In [ ]:
import csv
import json
#This program uses the CSV & JSON modules for reading/writing files
class Book(object):
    def __init__(self, id, title, author, genre, subgenre, publisher, loan = None, reserved=[]):
        self.id = id
        self.title = title
        self.genre = genre
        self.subgenre = subgenre
        self.pulisher = publisher
        self.loan = loan
        self.reserved = reserved
#Attributes of the class, includes the details from the CSV file as well as a loan attribute to show who currently has it loaned and a list of current reservations
    
    def scan(self):
        return ("ID: " + self.id + ", Title: " + self.title + ", Genre: " + self.genre + ", Subgenre: " + self.subgenre + ", Publisher: " + self.publisher)
#Scan function, returns all of a given books details
    
    def book_loan(self, member_no):
        self.loan = member_no
        for a in self.reserved:
            if a == member_no:
                self.reserved.remove(member_no)
#Sets the loan attribute to the members ID number, and removes the member from the reservation list if they're in it
    def book_reserved(self, member_no):
        if self.loan != None:
            self.reserved.append(member_no)
            print ("This book is currently reserved by" + self.reserved)
        else:
            print ("This book is not currently reserved")
#Function for book reservation alerts, when the member reaches index 0 of the reservation list it will print an alert, otherwise it will inform them of their place in the queue
    def loan_check(self):
        if self.loan is None:
            return False
        else:
            return True
#Checks whether a book is currently on loan, to ensure that members can't loan out books that aren't available
    def book_return(self, member_no):
        self.loan = None
#Returns the loan attribue back to None

class Member(object):
    def __init__(self, id, fname, sname, gender, email, card, loanedbooks=[]):
        self.id = id
        self.fname = fname
        self.sname = sname
        self.gender = gender
        self.email = email
        self.card = card
        self.loanedbooks = loanedbooks
#Attributes of the class, includes the details from the CSV file and a list for books the member has loaned        
    
    def scan(self):
        return ("id: " + self.id + ", First Name: " + self.fname + ", Second Name: " + self.sname + ", Gender: " + self.gender + ", Email: " + self.email  +", Card Number: " + self.card)
#Scan function for members, returns members personal details   
    
    def book_loan(self, bookid):
        self.loanedbooks.append(bookid)
#Function for loaning books, adds the book ID to the members loaned books list
    
    def book_return(self, bookid):
        self.loanedbooks.remove(bookid)
#Function for returning books, removes the book ID from the members loaned books list

def readcsv(file, type):
    instancelist = []
    with open(file, newline="") as csvfile:
        read = csv.reader(csvfile, delimiter = ",", quotechar="\"")
        next(read)
        for row in read:
            if type == "Book":
                instance = Book(*row)
            elif type == "Member":
                instance = Member(*row)
            else:
                exit()
            instancelist.append(instance)
    return instancelist
#Function to read CSV files that returns a list of books or members

def jsonwrite(file, booklist, memberlist):
    dict = {"Books":booklist, "Members":memberlist}
    with open(file, "w") as f:
        json.dump(dict, f, default = vars)
#Function to write to JSON        
        
def jsonread(file):
    with open(file, "r") as f:
        load = json.load(f)
    booklist = []
    memberlist = []
    
    for instance in load["Books"]:
        newbook = Book(**instance)
        booklist.append(newbook)
        
    for instance in load["Members"]:
        newmember = Member(**instance)
        memberlist.append(newmember)  
        
    return booklist, memberlist
#Function to Read from JSON files, requires the file name

def create_member():
    id = input("ID: ")
    fname = input("Please enter firstname: ")
    sname = input("Please enter second name: ")
    gender = input("Please enter gender: ")
    email = input("Please enter email: ")
    return Member(id, fname, sname, gender, email, int(id)*10+1)
#Function to create new membership applications, appends the new member to the members list when run

def sendemail(bookid, memberid):
    position = bookid.reservations.index(str(memberid))
    if position == 0:
        print ("This book is now available to be loaned")
    else:
        print("You a currently number %d in the reservation queue for this book" % (position))
#Function for sending updates on reservations

def library():
#This function requires the user to input the relevant data for the task they've decided: 
#Loaning and returning books require their member and book IDs
#Creating new members requires the personal information
#Saving a JSON requires a file name.
#Loaning and returning a book updates the relevant lists assigned in the book and member classes, 
#These changes are saved to the JSON file once option 4 is selected
    read = None
    while(read not in ["1", "2"]):
        read = input("Press 1 to load a CSV or 2 to load a JSON")
        if read == "1":
            bookfile = input("Enter book filename")
            try:
                books = readcsv(bookfile, "Book")
            except FileNotFoundError:
                print ("This file doesn't exist")
                read = None
                continue
            memberfile = input("Enter member file name")
            try:
                members = readcsv(memberfile, "Member")
            except FileNotFoundError:
                print ("This file doesn't exist")
                read = None
                continue
        elif read =="2":
            jsonfile = input("Enter filename")
            try:
                books, members = jsonread(jsonfile)
            except FileNotFoundError:
                print ("This file doesn't exist")
                read = None
                continue
        else:
            print("This isn't supported")
#Prompts users to enter a file to read from
#If CSV is selected it will ask for a book and a member file
#If JSON is selected it will ask for one file as all data is saved to a single file
    finished = False
        
    while not finished:
        function = input("Press 1 to loan a book, 2 to return a book, 3 to apply for a membership or 4 to save and exit")
        if function =="1":
            bookid = input("Please enter the book ID")
            memberid = input("Please enter your member ID")
            tempbooks = [a for a in range(len(books)) if books[a].id ==bookid]
            tempmembers = [a for a in range(len(members)) if members[a].id == memberid]
            if (len(tempbooks) == 0):
                print("This book doesn't exist")
                continue
            elif (len(tempmembers) == 0):
                print("This member ID doesn't exist")
                continue
            elif not books[tempbooks[0]].loan_check():
                books[tempbooks[0]].book_loan(memberid)
                members[tempmembers[0]].book_loan(bookid)
                print("Book has been loaned")
            else:
                print("This book is currently on loan, you have been added to the reservation list")
                books[tempbooks[0]].reserved.append(memberid)
#The main function for loaning books, requires the member and book ID number and will update the relevant lists when run 
        elif function == "2":
            bookid = input("Please enter the book ID")
            memberid = input("Please enter your member ID")
            tempbooks = [a for a in range(len(books)) if books[a].id ==bookid]
            tempmembers = [a for a in range(len(members)) if members[a].id == memberid]
            if (len(tempbooks) == 0):
                print("This book doesn't exist")
                continue
            elif (len(tempmembers) == 0):
                print("This member ID doesn't exist")
                continue
            else:
                books[tempbooks[0]].book_return(bookid)
                members[tempmembers[0]].book_return(bookid)
                print ("Thank you for returning this book")
#The main function for returning books, requires the member and book ID number and will update the relevant lists when run                
        elif function == "3":
            members.append(create_member())
#The main function for applying for membership, requires the members information and will append them to the members list when finished        
        elif function == "4":
            jsonname = input("Enter the name for the new JSON file")
            jsonwrite(jsonname, members, books)
            finished = True
#The main function for saving and exiting            
library()

The book class contains attributes for all of the books details, as well as the methods used by the program to check whether books are available, loan them and return them

The member class includes all of the members details as well as methods for updating their loaned book record

The create member function is used to apply for memberships, users input relevant data and the card number is worked out automatically, it appends the member list when finished

The send email function prints an alert if the member is at index 0 of the reservation list, if they're not at 0 it will inform them of their position in the queue

The main function lets the user choose from a variety of options, first they can choose to read from a csv or json file once this is selected they can chose from the following options; 
1)The main function for loaning books, members are prompted to enter their ID and the book ID, if the book is currently on loan they will be added to the reservation queue, if not it will be added to the users loaned books list. If the user was on the reservation list before they loaned the book they will be removed from it 
2)The main function for returning books, this will remove the book from the users loaned books list 
3)The main function for applying for memberships, this will append the members list with the details they are asked to input 4)The function to save and exit, this will prompt the user to input a new filename, save it as a JSON then exit the program